In [9]:
def get_lda_model():
    """
    LDA 모델과 토크나이저 로딩
    """
    count_vectorizer = joblib.load(str("../../modelapi/models/count_vectorizer.pkl"))
    lda_model = joblib.load(str("../../modelapi/models/best_lda_model.pkl"))

    with open(str("../../modelapi/db/stopwords-ko.txt"), "r", encoding="utf-8") as f:
        stopwords = [word.strip() for word in f.readlines()]

    return lda_model, count_vectorizer, stopwords

In [10]:
lda_model, count_vectorizer, stopwords = get_lda_model()

In [ ]:
import joblib
import re
from konlpy.tag import Okt
import pandas as pd
import numpy as np


def predict_topic_for_df(text):
  # 2. 형태소 분석기 및 불용어 로드
  okt = Okt()

  # 3. 텍스트 정제 함수
  def clean_text(text):
      text = re.sub(r"\[.*?\]|\(.*?\)", "", text)
      text = re.sub(r"[^가-힣\s]", "", text)
      text = re.sub(r"\s+", " ", text).strip()
      return text

  # 4. 명사 추출 함수
  def extract_nouns(text):
      nouns = okt.nouns(text)
      nouns = [word for word in nouns if word not in stopwords and len(word) > 1]
      return " ".join(nouns)

  # 5. 데이터 전처리 (정제 + 명사 추출)
  processed_texts = [extract_nouns(clean_text(text))]

  # 6. 벡터라이즈 (DTM 생성)
  new_dtm = count_vectorizer.transform(processed_texts)

  # 7. LDA 토픽 분포 예측
  topic_distribution = lda_model.transform(new_dtm)

  results = {}
  for index, value in enumerate(topic_distribution[0]):
    results[f'topic {index}'] = value

  return results

In [27]:
text = "선박 보수 서비스 기업인 HD현대마린솔루션이 2대 주주의 블록딜 소식에 주가가 약세를 보인 가운데, 모회사 HD현대로 지분율은 55.32%인 KKR의 지분율이 종전 19.52%에서 약 10%로 줄어들 전망이다."
predict_topic_for_df(text)

{'토픽 0': 0.017978642614813408,
 '토픽 1': 0.03786049083818201,
 '토픽 2': 0.019298977819376274,
 '토픽 3': 0.2511187593938351,
 '토픽 4': 0.019388808093020085,
 '토픽 5': 0.02491121925628149,
 '토픽 6': 0.30465217178799764,
 '토픽 7': 0.0752836697672914,
 '토픽 8': 0.24950726042920263}

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14087 entries, 0 to 14086
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            14087 non-null  int64  
 1   news_id               14087 non-null  object 
 2   wdate                 14087 non-null  object 
 3   title                 14087 non-null  object 
 4   article               14087 non-null  object 
 5   press                 14087 non-null  object 
 6   url                   14087 non-null  object 
 7   image                 14087 non-null  object 
 8   article_preprocessed  14087 non-null  object 
 9   summary               14087 non-null  object 
 10  stock_list            14087 non-null  object 
 11  industry_list         14087 non-null  object 
 12  summary_embedding     14087 non-null  object 
 13  기사 번호                 14087 non-null  int64  
 14  주요 토픽                 14087 non-null  object 
 15  토픽 1               

In [30]:
df = df[
    [
      "news_id",
      "토픽 1",
      "토픽 2",
      "토픽 3",
      "토픽 4",
      "토픽 5",
      "토픽 6",
      "토픽 7",
      "토픽 8",
      '토픽 9'
    ]
]

In [ ]:
df.rename(
    {
        "토픽 1": "topic_1",
        "토픽 2": "topic_2",
        "토픽 3": "topic_3",
        "토픽 4": "topic_4",
        "토픽 5": "topic_5",
        '토픽 6': 'topic_6', 
        '토픽 7': 'topic_7', 
        '토픽 8': 'topic_8', 
        '토픽 9': 'topic_9'
    }
)

In [28]:
df = pd.read_csv('../../db/news_2023_2025_tm.csv')
df.head()

,Unnamed: 0,news_id,wdate,title,article,press,url,image,article_preprocessed,summary,...,주요 토픽,토픽 1,토픽 2,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7,토픽 8,토픽 9
0,0,20250523_0002,2025-05-23 18:52:00,"[단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈",매각가 5000억 안팎 달할듯\r\n결제시장 내 입지강화 포석\r\n카카오페이 [사...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,매각가 5000억 안팎 달할듯 결제시장 내 입지강화 포석 카카오페이 국내 대표 전...,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,...,토픽 4,0.0122,0.0119,0.0121,0.5421,0.3730,0.0122,0.0120,0.0122,0.0122
1,1,20250523_0004,2025-05-23 18:33:00,"골드만삭스 차기 CEO, 이재용·김병주·이창용 만났다",[사진=연합뉴스]\r\n세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,...,토픽 9,0.0097,0.0096,0.0097,0.1966,0.0104,0.2332,0.0098,0.1834,0.3376
2,2,20250523_0007,2025-05-23 18:00:00,[단독] 전자결제 강자 카카오페이 쓱·스마일 페이 인수 추진,국내 대표 전자결제사업자인 카카오페이가 신세계이마트 산하 간편결제사업부 인수에 나섰...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...,국내 대표 전자결제사업자인 카카오페이가 신세계이마트 산하 간편결제사업부 인수에 나섰...,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,...,토픽 4,0.0114,0.0114,0.0116,0.5937,0.3260,0.0114,0.0113,0.0115,0.0118
3,3,20250523_0010,2025-05-23 17:52:00,조현준 효성重 지분 4.9% 美 테크펀드 2600억 매각,조현준 효성 회장이 상속세 재원 마련을 위해 효성중공업 지분 4.90%를 미국의 글...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...,효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중공업 주식...,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,...,토픽 9,0.0146,0.0145,0.0147,0.1025,0.1528,0.0145,0.0945,0.0147,0.5772
4,4,20250523_0011,2025-05-23 17:52:00,몸집 키우는 카카오…'간편결제 빅3' 흔드나,"카카오페이, 신세계 페이 품나\r\n이커머스 결제 강화 점유 확대\r\n압도적 1위...",매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,"카카오페이, 신세계 페이 품나 이커머스 결제 강화 점유 확대 압도적 1위 네이버에 ...",카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,...,토픽 4,0.1741,0.0130,0.0969,0.4690,0.0131,0.0132,0.0128,0.1948,0.0130
